# CSI AI Planner - Logic Verification Notebook

This notebook contains the core logic for the **AI Construction Planner** project. 
You can run these cells to verify the calculation logic, parsing, and rates independently of the Flask backend.

In [1]:
import math

# ==========================================
# 1. RATES & CONFIGURATION
# ==========================================
RATES = {
    "ar": {
        "carpentry_m2_per_day_per_carpenter": 15,
        "rebar_ton_per_day_per_steel_fixer": 0.8,
        "concrete_m3_per_hour_pump": 20,
        "crew_defaults": {
            "excavation": "حفار + 2 عمّال",
            "formwork": "4 نجّارين + 2 عمّال",
            "rebar": "3 حدادين + 2 عمّال",
            "pour": "فريق صب 6 عمّال",
            "curing": "2 عمّال"
        },
        "equipment_defaults": {
            "excavation": "حفّار + قلابات",
            "formwork": "معدات يدويّة",
            "rebar": "معدات حدادة",
            "pour": "خلاطة + مضخة",
            "curing": "رشّ مياه"
        }
    },
    "en": {
        "carpentry_m2_per_day_per_carpenter": 15,
        "rebar_ton_per_day_per_steel_fixer": 0.8,
        "concrete_m3_per_hour_pump": 20,
        "crew_defaults": {
            "excavation": "Excavator + 2 laborers",
            "formwork": "4 carpenters + 2 laborers",
            "rebar": "3 steel fixers + 2 laborers",
            "pour": "Pouring crew 6 laborers",
            "curing": "2 laborers"
        },
        "equipment_defaults": {
            "excavation": "Excavator + trucks",
            "formwork": "Hand tools",
            "rebar": "Rebar tools",
            "pour": "Mixer + pump",
            "curing": "Water spray"
        }
    }
}

In [4]:
# ==========================================
# 2. HELPER FUNCTIONS
# ==========================================

def parse_query(q):
    # Very simple parser: detect quantity and unit (m3), and keywords
    q = q.lower()
    qty = None
    unit = None
    # Quantity
    for token in q.replace("م٣","م3").replace("cubic","m3").replace("cum","m3").split():
        if "m3" in token or "م3" in token:
            # try to read number just before
            try:
                qty = float(''.join([c for c in token if c.isdigit() or c=='.']))
                unit = "m3"
            except:
                pass
    # Fallback: search number
    if qty is None:
        nums = ''.join([c if c.isdigit() or c=='.' else ' ' for c in q]).split()
        if nums:
            try:
                qty = float(nums[0])
            except:
                qty = None

    # Keywords
    if any(k in q for k in ["raft", "mat", "labsha", "لبشة", "حصيرية"]):
        scope = "raft_foundation"
    elif any(k in q for k in ["foundation", "foundations", "أساسات", "قواعد", "isolated"]):
        scope = "isolated_foundations"
    else:
        scope = "generic_concrete"

    return qty, unit or "m3", scope

In [5]:
# ==========================================
# 3. PLANNING LOGIC
# ==========================================

def plan_isolated_foundations(qty_m3, lang):
    L = RATES[lang]
    rows = []

    # Assumptions (Isolated: High Formwork Ratio)
    formwork_area = qty_m3 * 1.2
    rebar_ton = qty_m3 * 0.08
    
    carp_rate = L["carpentry_m2_per_day_per_carpenter"] * 4
    rebar_rate = L["rebar_ton_per_day_per_steel_fixer"] * 3
    pump_rate_m3_per_day = L["concrete_m3_per_hour_pump"] * 6

    # Excavation
    exc_days = math.ceil(qty_m3 / 40.0)
    rows.append({
        "task": "حفر" if lang == "ar" else "Excavation",
        "qty": f"{qty_m3:.1f}",
        "unit": "م³" if lang == "ar" else "m3",
        "crew": L["crew_defaults"]["excavation"],
        "equipment": L["equipment_defaults"]["excavation"],
        "duration": f"{exc_days} { 'يوم' if lang=='ar' else 'day' }"
    })

    # Formwork
    form_days = math.ceil(formwork_area / carp_rate)
    rows.append({
        "task": "نجارة شدّات" if lang == "ar" else "Formwork",
        "qty": f"{formwork_area:.1f}",
        "unit": "م²" if lang == "ar" else "m2",
        "crew": L["crew_defaults"]["formwork"],
        "equipment": L["equipment_defaults"]["formwork"],
        "duration": f"{form_days} { 'يوم' if lang=='ar' else 'day' }"
    })

    # Rebar
    rebar_days = math.ceil(rebar_ton / rebar_rate)
    rows.append({
        "task": "حدادة مسلّحة" if lang == "ar" else "Rebar",
        "qty": f"{rebar_ton:.2f}",
        "unit": "طن" if lang == "ar" else "ton",
        "crew": L["crew_defaults"]["rebar"],
        "equipment": L["equipment_defaults"]["rebar"],
        "duration": f"{rebar_days} { 'يوم' if lang=='ar' else 'day' }"
    })

    # Pouring
    pour_days = math.ceil(qty_m3 / pump_rate_m3_per_day)
    rows.append({
        "task": "صب خرسانة" if lang == "ar" else "Concrete Pour",
        "qty": f"{qty_m3:.1f}",
        "unit": "م³" if lang == "ar" else "m3",
        "crew": L["crew_defaults"]["pour"],
        "equipment": L["equipment_defaults"]["pour"],
        "duration": f"{pour_days} { 'يوم' if lang=='ar' else 'day' }"
    })

    # Curing
    cure_days = 7
    rows.append({
        "task": "معالجة خرسانة" if lang == "ar" else "Curing",
        "qty": f"{qty_m3:.1f}",
        "unit": "م³" if lang == "ar" else "m3",
        "crew": L["crew_defaults"]["curing"],
        "equipment": L["equipment_defaults"]["curing"],
        "duration": f"{cure_days} { 'يوم' if lang=='ar' else 'day' }"
    })

    # Overall text
    if lang == "ar":
        text = (
            "تم توليد خطة للأساسات المنفصلة:\n"
            f"- الكمية: {qty_m3} م³\n"
            f"- تقدير شدّات: {formwork_area:.1f} م²\n"
            f"- تقدير حديد: {rebar_ton:.2f} طن\n"
            f"- المدة التقريبية: {exc_days + form_days + rebar_days + pour_days + cure_days} يوم"
        )
        notes = "معدلات تقريبية للأساسات المنفصلة."
    else:
        text = (
            "Generated plan for Isolated Foundations:\n"
            f"- Quantity: {qty_m3} m3\n"
            f"- Formwork: {formwork_area:.1f} m2\n"
            f"- Rebar: {rebar_ton:.2f} ton\n"
            f"- Approx Duration: {exc_days + form_days + rebar_days + pour_days + cure_days} days"
        )
        notes = "Approximate rates for isolated footings."

    return {
        "text": text,
        "table": { "rows": rows },
        "notes": notes
    }

def plan_raft_foundation(qty_m3, lang):
    L = RATES[lang]
    rows = []

    # Assumptions (Raft: Lower Formwork Ratio, Higher Rebar)
    # Raft is mostly volume, less side area.
    formwork_area = qty_m3 * 0.4  # significantly less per m3 than isolated
    rebar_ton = qty_m3 * 0.10     # often heavier reinforcement (100kg/m3)
    
    carp_rate = L["carpentry_m2_per_day_per_carpenter"] * 4
    rebar_rate = L["rebar_ton_per_day_per_steel_fixer"] * 3
    pump_rate_m3_per_day = L["concrete_m3_per_hour_pump"] * 8  # Long pour days for rafts

    # Excavation (Mass excavation)
    exc_days = math.ceil(qty_m3 / 80.0) # Faster (larger equipment)
    rows.append({
        "task": "حفر لبشة" if lang == "ar" else "Mass Excavation",
        "qty": f"{qty_m3:.1f}",
        "unit": "م³" if lang == "ar" else "m3",
        "crew": L["crew_defaults"]["excavation"],
        "equipment": L["equipment_defaults"]["excavation"],
        "duration": f"{exc_days} { 'يوم' if lang=='ar' else 'day' }"
    })

    # Formwork
    form_days = math.ceil(formwork_area / carp_rate)
    rows.append({
        "task": "نجارة جوانب اللبشة" if lang == "ar" else "Raft Side Formwork",
        "qty": f"{formwork_area:.1f}",
        "unit": "م²" if lang == "ar" else "m2",
        "crew": L["crew_defaults"]["formwork"],
        "equipment": L["equipment_defaults"]["formwork"],
        "duration": f"{form_days} { 'يوم' if lang=='ar' else 'day' }"
    })

    # Rebar (Takes longer usually)
    rebar_days = math.ceil(rebar_ton / rebar_rate)
    rows.append({
        "task": "حدادة اللبشة" if lang == "ar" else "Raft Rebar",
        "qty": f"{rebar_ton:.2f}",
        "unit": "طن" if lang == "ar" else "ton",
        "crew": L["crew_defaults"]["rebar"],
        "equipment": L["equipment_defaults"]["rebar"],
        "duration": f"{rebar_days} { 'يوم' if lang=='ar' else 'day' }"
    })

    # Pouring (Massive pour)
    pour_days = math.ceil(qty_m3 / pump_rate_m3_per_day)
    rows.append({
        "task": "صب خرسانة اللبشة" if lang == "ar" else "Raft Concrete Pour",
        "qty": f"{qty_m3:.1f}",
        "unit": "م³" if lang == "ar" else "m3",
        "crew": "2x " + L["crew_defaults"]["pour"], # Double crew for raft
        "equipment": "2x " + L["equipment_defaults"]["pour"],
        "duration": f"{pour_days} { 'يوم' if lang=='ar' else 'day' }"
    })

    # Curing
    cure_days = 14 # Longer curing for mass concrete
    rows.append({
        "task": "معالجة (غمْر بالمياه)" if lang == "ar" else "Curing (Ponding)",
        "qty": f"{qty_m3:.1f}",
        "unit": "م³" if lang == "ar" else "m3",
        "crew": L["crew_defaults"]["curing"],
        "equipment": L["equipment_defaults"]["curing"],
        "duration": f"{cure_days} { 'يوم' if lang=='ar' else 'day' }"
    })

    # Overall text
    if lang == "ar":
        text = (
            "تم توليد خطة للبشة الخرسانية (Raft):\n"
            f"- الكمية: {qty_m3} م³\n"
            f"- مساحة الجوانب التقديرية: {formwork_area:.1f} م²\n"
            f"- حديد التسليح التقديري (100كجم/م³): {rebar_ton:.2f} طن\n"
            f"- المدة الزمنية: {exc_days + form_days + rebar_days + pour_days + cure_days} يوم"
        )
        notes = "تم افتراض صب اللبشة باستخدام مضختين وفريقين لضمان الاستمرارية."
    else:
        text = (
            "Generated plan for Raft Foundation (Mat):\n"
            f"- Quantity: {qty_m3} m3\n"
            f"- Side Formwork: {formwork_area:.1f} m2\n"
            f"- Rebar (approx 100kg/m3): {rebar_ton:.2f} ton\n"
            f"- Duration: {exc_days + form_days + rebar_days + pour_days + cure_days} days"
        )
        notes = "Assumed usage of 2 concrete pumps and double crew for massive pour continuity."

    return {
        "text": text,
        "table": { "rows": rows },
        "notes": notes
    }

In [6]:
# ==========================================
# 4. TEST SCENARIOS
# ==========================================

def ai_simulator(query, lang="en"):
    print(f"--- User Query: {query} ---")
    qty, unit, scope = parse_query(query)
    print(f"Parsed: Qty={qty}, Unit={unit}, Scope={scope}")
    
    if scope == "isolated_foundations" and qty:
        return plan_isolated_foundations(qty, lang)
    elif scope == "raft_foundation" and qty:
        return plan_raft_foundation(qty, lang)
    else:
        return "Could not parse or unsupported scope."

# Scenario 1: Isolated Footings
res1 = ai_simulator("Isolated foundations 100 m3 concrete", "en")
print(res1)

# Scenario 2: Raft Foundation (Arabic)
res2 = ai_simulator("لبشة خرسانية 120 م3", "ar")
print(res2)

# Scenario 3: Raft Foundation (English shorthand)
res3 = ai_simulator("120 cum raft", "en")
print(res3)

--- User Query: Isolated foundations 100 m3 concrete ---
Parsed: Qty=3.0, Unit=m3, Scope=isolated_foundations
{'text': 'Generated plan for Isolated Foundations:\n- Quantity: 3.0 m3\n- Formwork: 3.6 m2\n- Rebar: 0.24 ton\n- Approx Duration: 11 days', 'table': {'rows': [{'task': 'Excavation', 'qty': '3.0', 'unit': 'm3', 'crew': 'Excavator + 2 laborers', 'equipment': 'Excavator + trucks', 'duration': '1 day'}, {'task': 'Formwork', 'qty': '3.6', 'unit': 'm2', 'crew': '4 carpenters + 2 laborers', 'equipment': 'Hand tools', 'duration': '1 day'}, {'task': 'Rebar', 'qty': '0.24', 'unit': 'ton', 'crew': '3 steel fixers + 2 laborers', 'equipment': 'Rebar tools', 'duration': '1 day'}, {'task': 'Concrete Pour', 'qty': '3.0', 'unit': 'm3', 'crew': 'Pouring crew 6 laborers', 'equipment': 'Mixer + pump', 'duration': '1 day'}, {'task': 'Curing', 'qty': '3.0', 'unit': 'm3', 'crew': '2 laborers', 'equipment': 'Water spray', 'duration': '7 day'}]}, 'notes': 'Approximate rates for isolated footings.'}
---